In [49]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf

import os
from pathlib import Path

## Tomato Predictor - An introduction
The idea of this project is to learn about:

1. Convolutional Neural Networks (CNNs)
    - Convolution and pooling
    - Common arquitectures - LeNet, AlexNet, VGG, ResNet

2. Image Preprocessing
    - Normalization and standarization of pixels
    - Data Augmentation: from the same sample, generate more images
    - Noise reduction and constrast enhancement

3. Evaluation metrics
    - Precission, recall, F1-score
    - Confusion matrix
    - ROC and AUC curve

4. Optimization and regularization
    - Optimization algorithm - SGC, Adam, RMSprop
    - Techniques to avoid overfitting - dropout, batch normalization, early stopping

## Data

### Extracting data from directories

In [80]:
from pathlib import Path

route_train = 'content/ieee-mbl-cls/train'
route_test = 'content/ieee-mbl-cls/val'
 
dynamic_route_train = "{}/{}".format(route_train, "{}")
dynamic_route_test = "{}/{}".format(route_test, "{}")

labels = []
allowed_extension = ".jpg"

def extract_labels(route: str) -> None:
    pathlist = Path(route).iterdir()
    for path in pathlist:
        # because path is object not string
        label = str(path.name)   
        labels.append(label)

def convert_data_to_df(dynamic_route):
    images = {}  # dict with "image_key_name": "label"

    for label in labels:
        path = Path(dynamic_route.format(label))

        for img_path in path.iterdir():
            ext = img_path.suffix.lower()
            if ext != allowed_extension:
                continue
            
            images[img_path.name] = img_path.name[0]

    df = pd.DataFrame({
        "name": list(images.keys()),
        "label": list(images.values()), 
    })
    
    return df
    
extract_labels(route_train)

In [81]:
df = convert_data_to_df(dynamic_route_train)
df = df.sample(frac=1).reset_index(drop=True) # shuffle the dataframe
df.head()

,name,label
0,o (1420).jpg,o
1,r (1110).jpg,r
2,u (1188).jpg,u
3,r (466).jpg,r
4,u (189).jpg,u


In [82]:
df_test = convert_data_to_df(dynamic_route_test)
df_test = df_test.sample(frac=1).reset_index(drop=True) # shuffle the dataframe
df_test.head()

,name,label
0,r (989).jpg,r
1,u (978).jpg,u
2,o (426).jpg,o
3,u (712).jpg,u
4,d (397).jpg,d


### From images to pixels

Our images are a 256px x 256px. Then, we have adjust our CNN for this. However, we first have to extract the pixel information of the image in order to be able to process the data in the neural network.

In [90]:
import pandas as pd
from PIL import Image
from numpy import asarray

def add_pixels_image_data(df, route, index, name, target_size=(256, 256)):
    """
    For the row at the specified index, it will append the image's pixel data after processing it.
    The image will be resized and normalized before being added to the DataFrame.
    """
    first = name[0]
    
    if first == 'r':
        first = "Ripe"
    elif first == "o":
        first = "Old"
    elif first == "u":
        first = "Unripe"
    else:
        first = "Damaged"

    path = f"{route}/{first}/{name}"
    image = Image.open(path)
    print(f"Format: {image.format}, pixels: {image.size}, mode: {image.mode}\n")    
    image = image.resize(target_size)

    numpydata = asarray(image)
    numpydata = numpydata.astype('float32')

    if 'image' not in df.columns:
        df['image'] = pd.Series([None] * len(df))

    df.at[index, 'image'] = numpydata / 255 # normalise data [0, 1]

    return df

In [ ]:
for index, row in df.iterrows():
    df = add_pixels_image_data(df, route_train, index, row['name'])

In [55]:
df.head()

,name,label,image
0,r (626).jpg,r,"[[[0.7647059, 0.77254903, 0.7529412], [0.76862..."
1,o (652).jpg,o,"[[[0.78039217, 0.7882353, 0.7764706], [0.78431..."
2,u (248).jpg,u,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,..."
3,r (1144).jpg,r,"[[[0.79607844, 0.8, 0.78039217], [0.79607844, ..."
4,o (1914).jpg,o,"[[[0.78431374, 0.76862746, 0.7647059], [0.7843..."


In [56]:
df.shape

(5832, 3)

In [57]:
df.at[0, "image"].shape

(256, 256, 3)

In [59]:
for i, img in enumerate(df['image']):
    if img.shape != ((256, 256, 3)):
        df = df.drop(index=i)

In [60]:
df.shape

(5782, 3)

In [70]:
X_train = np.array(df['image'].tolist())
Y_train = np.array(df["label"].tolist())

print(X_train.shape)
print(Y_train.shape)

(5782, 256, 256, 3)
(5782,)


### For the validation images

In [ ]:
for index, row in df_test.iterrows():
    df_test = add_pixels_image_data(df_test, route_test, index, row['name'])

In [93]:
for i, img in enumerate(df_test['image']):
    if img.shape != ((256, 256, 3)):
        df_test = df_test.drop(index=i)

In [95]:
X_val = np.array(df_test["image"].tolist())
Y_val = np.array(df_test["label"].tolist())

print(X_val.shape)
print(Y_val.shape)

(653, 256, 256, 3)
(653,)


## Basic Neural Network

### How to determine the number of nodes and layers?

Funny enough, I found this in StackOverflow: [Publication](https://stackoverflow.com/questions/35520587/how-to-determine-the-number-of-layers-and-nodes-of-a-neural-network).

---
For the **layers**, just keep adding layers until the test error does not improve anymore.

For the **nodes**, you should have one node per feature (in the input layer), which makes sense. In this case, we only have one feature, but we can convert that list of `196608` pixels $\rightarrow$ `196608` nodes... But this might not be optimal, so better be to not flatten that image.

---
When selecting the nodes, its important to know that:
- CNN: we maintain the original structure of the image (x, y, RGB#)
- Dense: we must flatten the image data `.flatten()`


In [72]:
from tensorflow.keras import models
from tensorflow.keras import layers

In [73]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

In [74]:
model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_4 (Conv2D)               │ (None, 254, 254, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 127, 127, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 125, 125, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 62, 62, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 60, 60, 64)     │        36,928 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 56,320 (220.00 KB)

 Trainable params: 56,320 (220.00 KB)

 Non-trainable params: 0 (0.00 B)

In [77]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(4, activation="softmax"))

In [78]:
model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_4 (Conv2D)               │ (None, 254, 254, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 127, 127, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 125, 125, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 62, 62, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 60, 60, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 230400)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │    14,745,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 4)              │           260 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,806,404 (56.48 MB)

 Trainable params: 14,806,404 (56.48 MB)

 Non-trainable params: 0 (0.00 B)

In [101]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

Y_train_encoded = label_encoder.fit_transform(Y_train)
Y_val_encoded = label_encoder.transform(Y_val)

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x108119910>>
Traceback (most recent call last):
  File "/Users/idb0123/Desktop/tomato-predictor/.venv/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


KeyboardInterrupt: 

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(X_train, Y_train_encoded, epochs=10, 
                    validation_data=(X_val, Y_val_encoded))